In [72]:
from IPython.display import display, Math, Latex

* Let P(x) ~ x^2
* Normalize P(x): $\int_{-\inf}^{x_{max}}{\beta \cdot x^2} = 1$ $\Rightarrow$ $\beta = \frac{3}{x_{max}^3}$

In [126]:

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
import numpy as np                                                                    # 
from matplotlib import pyplot as plt                                                  #
from scipy import stats                                                               #
from astroML.stats.random import trunc_exp                                            #
from astroML.plotting import setup_text_plots                                         #
from scipy import integrate                                                           #
from scipy.optimize import fmin,leastsq,fminbound,basinhopping                        #
from scipy import optimize                                                            #
#%matplotlib inline                                                                   #
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#

#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
#                      STEP 1 - create our analytical function                        #
Npoints=100                                                                           #
a = 2.                                                                                #
x = np.linspace(0.01, 1, Npoints)                                                     #
P_x = 3./max(x)**3 * x**a                                                             #
                                                                                      #
#                    STEP 2 - create our intrinsic distribution                       #
intrinsic_distribution = (np.random.uniform(size=Npoints))**(1.0/3.0) * max(x)        # 
                                                                                      #
#                STEP 3 - create our mock data/observed distribution                  #
sigma=0.2                                                                             #
noise=np.random.normal(loc=0., scale=sigma, size=Npoints)                             #
observed_distribution = intrinsic_distribution + noise                                #
fig, ax= plt.subplots(figsize=(8, 10))                                                #
bins=np.linspace(min(observed_distribution), max(observed_distribution), 30)          #
ax.hist(intrinsic_distribution, histtype='step', color='k', linewidth=2., bins=bins, normed=True, label='true')
ax.hist(observed_distribution, histtype='step', color='b', linewidth=2., bins=bins, normed=True, label='observed')
ax.plot(x, P_x, "r--", linewidth=2)                                                   #
ax.legend(loc='upper left')                                                           #
plt.show()                                                                            #
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
binsize=bins[1]-bins[0]


Now we want to maximize:
$$ P(A|B) = \frac{P(B|A) P(A)}{P(B)} $$

or in other words minimize the negative logarithm of the expression above:
$$ -log(P(B|A)) -log(P(A)) $$




                           WHAT IS THE LIKELIHOOD??

* For arbitrary exponent a, P(x) ~$x^a$ is normalized as P(x) = $(a+1) \cdot x^a (\frac{1}{x_{max}^{a+1}} - \frac{1}{x_{min}^{a+1}})$ $\Rightarrow$ P(x) = $(a+1) \cdot x^a \frac{1}{x_{max}^{a+1}}$

since we integrate from $-\infty$ to $x_{max}$

* Convolved with (assumed) gaussian errors the likelihood is thus: 
P(B|A) = $\frac{1}{\sigma\cdot \sqrt{2 \cdot \pi}}$ $\int_{x_{min}}^{x_{max}}{P(x)\cdot e^{\frac{-(x-x')^2}{2\cdot \sigma}}} dx'$


In [139]:
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
#                So we need to evaluate the integral above for each point             #
#                                                                                     #
#                          Define integral function                                   #
def integral(xprime, xobs, a, sigma):                                                 #
    return (a+1) * (1.0/(max(x)**(a+1))) * xprime**a * np.exp(-(xobs-xprime)**2/(2*sigma**2))/(sigma*np.sqrt(2*np.pi))
#                                                                                     #
#                                                                                     #
#                        Define likelihood function                                   #
def calcLikelihood(a, observed_distribution, sigma):                                  #
                                                                                      #
    LikelihoodProduct=[]                                                              #
                                                                                      #
    for i in range (0, len(observed_distribution)):                                   #
                                                                                      #
        xobs=observed_distribution[i]                                                 #
                                                                                      #
        Likelihood_point, error_point = integrate.quad(integral, min(x), max(x), args=(xobs, a, sigma))
                                                                                      #
        LikelihoodProduct.append(Likelihood_point)                                    #
                                                                                      #
    LikelihoodProduct=np.array(LikelihoodProduct)                                     #
                                                                                      #
    LikelihoodProduct=np.log(LikelihoodProduct)                                       #
                                                                                      #
    return LikelihoodProduct.sum()                                                    #
                                                                                      #
#                          Define prior function                                      #
def prior(a):                                                                         #
    #   For a flat distribution  #                                                    #
    if a>100. and a<-100.:                                                            #
        return -np.inf                                                                #
    else:                                                                             #
        return 0.                                                                     #
                                                                                      #
#                            add the two (log space)                                  #
def multiply(a, observed_distribution, sigma):                                        #
    return -(calcLikelihood(a, observed_distribution, sigma) + prior(a))              #
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#


In [140]:
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
#                          Minimize the sum (minus the sum)                           #
a_initguess = -0.1                                                                    #
bounds = ((-0.99, 100.0),)                                                            #
xopt = optimize.minimize(multiply, a_initguess, args=(observed_distribution, sigma), method='TNC', bounds=bounds)
                                                                                      #
if xopt.success:                                                                      #
    print("Optimization successful. Power law exponent:", xopt.x)                     #
else:                                                                                 #
    print("Optimization failed.")                                                     #
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#

('Optimization successful. Power law exponent:', array([ 1.68969384]))


In [141]:
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-#
#                        How does the likelihood looks like???                        #
agrid=np.linspace(-20, 20, 100)                                                       #
L_func_a = []                                                                         #
for i in range (0, len(agrid)):                                                       #
    L_func_a.append(multiply(agrid[i], observed_distribution, sigma))                 #
plt.plot(agrid, L_func_a, linewidth=1.5, color='k')                                   #
plt.ylabel('Likelihood')                                                              #
plt.xlabel('exponent values')                                                         #
plt.show()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in log
